In [22]:
import pandas as pd
import regex as re
from textblob import TextBlob
from nltk.corpus import stopwords #stopwords
from nltk import pos_tag,pos_tag_sents #POS tags
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
#from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split  
from sklearn.model_selection import GridSearchCV 
#GridSearchCV-advanced form of cross-validation
import pickle
#BernoulliNB

In [67]:
data=pd.read_csv("spam.csv",encoding='ISO-8859-1')
#data=pd.read_csv("spam.csv",encoding='UTF-8')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 217.8+ KB


In [25]:
data.describe(include='all')

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""",GE,"GNT:-)"""
freq,4825,30,3,2,2


In [26]:
data.head(5)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 217.8+ KB


In [27]:
data['org_text']=data['v2']
data['org_text']

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: org_text, Length: 5572, dtype: object

In [28]:
#Convert to lower case
data['v2']=data['v2'].str.lower()

In [29]:
#Remove URL's
data['v2']=data['v2'].str.replace('http\S+|www.\S+','',case=False)

In [30]:
#Remove numbers
data['v2']=data['v2'].str.replace('\d+','')

In [31]:
#Remove whitespaces
data['v2']=data['v2'].str.replace('\W+',' ')

In [32]:
data['v2']=data['v2'].str.replace('\s{2,}',' ')
data['v2']=data['v2'].str.replace('\s{3,}',' ')

In [33]:
from textblob import TextBlob, Word

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n',
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]  
    #dictionary.get(keyname, value)
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [34]:
data['v2']=data.v2.apply(lemmatize_with_postag)

In [35]:
#data['v2']
stop_words_nltk=nltk.corpus.stopwords.words('english')
for word in stop_words_nltk :
    data['v2'] = data['v2'].replace(to_replace=r'\b%s\b'%word, value="",regex=True)

In [36]:
data['v2']=data['v2'].map(lambda x:x.strip())

In [37]:
def pos(text):
    return TextBlob(text).tags

In [38]:
pos_output=data.v2.apply(pos)

In [39]:
pos_output

0       [(go, VB), (jurong, JJ), (point, NN), (crazy, ...
1       [(ok, JJ), (lar, JJ), (joking, NN), (wif, NN),...
2       [(free, JJ), (entry, NN), (wkly, VBD), (comp, ...
3       [(u, JJ), (dun, NNS), (say, VBP), (early, JJ),...
4       [(nah, RB), (think, VBP), (go, VB), (usf, JJ),...
                              ...                        
5567    [(nd, JJ), (time, NN), (try, VB), (contact, NN...
5568    [(ì_, NN), (b, NN), (go, VBP), (esplanade, JJ)...
5569           [(pity, NN), (mood, NN), (suggestion, NN)]
5570    [(guy, NN), (bitching, VBG), (act, NN), (like,...
5571                 [(rofl, NN), (true, JJ), (name, NN)]
Name: v2, Length: 5572, dtype: object

In [40]:
pos_output=pd.DataFrame(pos_output)
pos_output

,v2
0,"[(go, VB), (jurong, JJ), (point, NN), (crazy, ..."
1,"[(ok, JJ), (lar, JJ), (joking, NN), (wif, NN),..."
2,"[(free, JJ), (entry, NN), (wkly, VBD), (comp, ..."
3,"[(u, JJ), (dun, NNS), (say, VBP), (early, JJ),..."
4,"[(nah, RB), (think, VBP), (go, VB), (usf, JJ),..."
...,...
5567,"[(nd, JJ), (time, NN), (try, VB), (contact, NN..."
5568,"[(ì_, NN), (b, NN), (go, VBP), (esplanade, JJ)..."
5569,"[(pity, NN), (mood, NN), (suggestion, NN)]"
5570,"[(guy, NN), (bitching, VBG), (act, NN), (like,..."


In [41]:
data['v2'] = pos_output['v2'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

In [42]:
review_train, review_test, label_train, label_test = train_test_split(data['v2'],data['v1'], test_size=0.2,random_state=100)

In [43]:
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True,sublinear_tf=False, ngram_range=(2,2)) #min_df=0.1)

In [44]:
X_train_tf=tf_vect.fit_transform(review_train)

In [46]:
X_train_tf=pd.DataFrame(X_train_tf.todense(),columns=tf_vect.get_feature_names())

In [47]:
X_test_tf = tf_vect.transform(review_test)
from sklearn.externals import joblib
joblib.dump(tf_vect,'tfidf_obj.sav')

In [48]:
X_test_tf=pd.DataFrame(X_test_tf.todense(),columns=tf_vect.get_feature_names())

In [49]:
X_test_tf

,____ nnp,aah jj,aah nn,aaniye vbp,aaooooright jj,aathi nn,abbey vbz,abdomen nns,abeg nns,abel nn,...,û_thanks nns,ûªm vbp,ûªt nnp,ûï nn,ûïharry nnp,ûò in,ûò jj,ûò nnp,ûò vbd,ûówell nnp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
X_train_tf

,____ nnp,aah jj,aah nn,aaniye vbp,aaooooright jj,aathi nn,abbey vbz,abdomen nns,abeg nns,abel nn,...,û_thanks nns,ûªm vbp,ûªt nnp,ûï nn,ûïharry nnp,ûò in,ûò jj,ûò nnp,ûò vbd,ûówell nnp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train_tf, label_train)
joblib.dump(model,'Gaussian_obj.sav')

['Gaussian_obj.sav']

In [53]:
pred_Naive=model.predict(X_test_tf)

In [54]:
metrics.accuracy_score(label_test, pred_Naive)

0.9004484304932735

In [54]:
from sklearn import svm
def svc_param(x, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(x, y)
    return grid_search.best_params_

In [ ]:
#svc_param(X_train_tf, label_train,2)

In [58]:
from sklearn import svm
model_svc=svm.SVC(kernel='linear',C=1,gamma=0.001)
model_svc.fit(X_train_tf, label_train)
joblib.dump(model_svc,'SVC_obj.sav')

['SVC_obj.sav']

In [59]:
label_pred=model_svc.predict(X_test_tf)

In [60]:
metrics.accuracy_score(label_test, label_pred)

0.9847533632286996

In [61]:
from sklearn.ensemble import RandomForestClassifier
def randomforest_param(x, y, nfolds):
    n_estimator_val = [100,150,300,500,1000]
    n_sample_leaf_val = [1,2,3,4,5,6]
    max_feature_val=["auto","sqrt",None,0.9]
    param_grid = {'n_estimators': n_estimator_val, 'min_samples_leaf' : n_sample_leaf_val,'max_features':max_feature_val}
    grid_search = GridSearchCV(RandomForestClassifier, param_grid, cv=nfolds)
    grid_search.fit(x, y)
    return grid_search.best_params_

In [ ]:
#randomforest_param(X_train_tf, label_train,2)

In [63]:
model_random=RandomForestClassifier(n_estimators=1000, 
                              oob_score=True, 
                              n_jobs=-1,  
                              max_features="auto", 
                              min_samples_leaf=5)
model_random.fit(X_train_tf, label_train)
joblib.dump(model_random,'Randomforest_obj.sav')

['Randomforest_obj.sav']

In [64]:
label_pred=model_random.predict(X_test_tf)

In [65]:
metrics.accuracy_score(label_test, label_pred)

0.9721973094170404